In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [67]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(777)

In [68]:
Fold = 5

# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = '/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/Training_Fold%d'%(i+1)
    path2 = '/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/Validation_Fold%d'%(i+1)
    c1 = 'Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/TrainingFold_Label_forANN'  , sep=",", header=None).T.squeeze())
ValidationFold_Label = np.array(pd.read_csv('/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/ValidationFold_Label_forANN', sep=",", header=None).T.squeeze())
    
    
# 전체 학습용 데이터
Training_All       = np.array(pd.read_csv('/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('/content/drive/My Drive/online_shoppers_intention.csv/K_FoldData/Training_All_Label_forANN', sep = ",", header = None).T.squeeze())

TrainingFold_Label=np.transpose(TrainingFold_Label)
ValidationFold_Label=np.transpose(ValidationFold_Label)

print(Training_Fold1.shape)
print(Validation_Fold1.shape)
print(TrainingFold_Label.shape)
print(ValidationFold_Label.shape)
print(Training_All.shape)

(336, 14)
(84, 14)
(336, 2)
(84, 2)
(420, 14)


In [72]:
learningRate  = 0.001
noOfNeuron    = 20
iteration     = 10000

In [73]:
# input place holders
X = tf.placeholder(tf.float32, [None, Training_Fold1.shape[1]])
Y = tf.placeholder(tf.float32, [None, 2])

# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers

# Input Layer
W1 = tf.Variable(tf.random_normal([Training_Fold1.shape[1], noOfNeuron]))
b1 = tf.Variable(tf.random_normal([noOfNeuron]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

# Hidden Layer 1
W2 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b2 = tf.Variable(tf.random_normal([noOfNeuron]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

# Hidden Layer 2
W3 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b3 = tf.Variable(tf.random_normal([noOfNeuron]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

# Output Layer
W4 = tf.Variable(tf.random_normal([noOfNeuron, 2]))
b4 = tf.Variable(tf.random_normal([2]))
hypothesis = tf.matmul(L3, W4) + b4


# define cost/loss &optimizer
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [95]:
Label = TrainingFold_Label
Label_Val = ValidationFold_Label

for i in range(Fold):
    
    s1= 'Data = Training_Fold%d'%(i+1)
    s2= 'Data_Val = Validation_Fold%d'%(i+1)
    exec(s1)
    exec(s2)

# train my model
# initialize
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for step in range(1, iteration+1):
    
        c, _ = sess.run([cost, optimizer], feed_dict = {X : Data, Y: Label})
        s2 = 'Validation_Acc_Fold%d = sess.run(accuracy, feed_dict={X : Data_Val, Y: Label_Val})'%(i+1)
        exec(s2)

Accuracy_sum = 0

for j in range(Fold):
    
    s3 = 'Accuracy_sum += Validation_Acc_Fold%d'%(j+1)
    exec(s3)
    
print('[Reslut of K-fold Cross Validation] \n')
print('Fold 1: {:.5f} \nFold 2: {:.5f} \nFold 3: {:.5f} \nFold 4: {:.5f} \nFold 5: {:.5f} \n\n * Average accuracy : {:.5f}'
      .format(Validation_Acc_Fold1, Validation_Acc_Fold2, Validation_Acc_Fold3, Validation_Acc_Fold4, Validation_Acc_Fold5, Accuracy_sum/Fold))

[Reslut of K-fold Cross Validation] 

Fold 1: 0.94048 
Fold 2: 0.94048 
Fold 3: 0.91667 
Fold 4: 0.97619 
Fold 5: 0.95238 

 * Average accuracy : 0.94524


In [96]:
x_data = Training_All
y_data =np.transpose(Training_All_Label) 

In [97]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(1, iteration+1):
    
    c, _ = sess.run([cost, optimizer], feed_dict = {X : x_data, Y: y_data})
    Acc = sess.run(accuracy, feed_dict={X : x_data, Y: y_data})
   
    if step % 1000 == 0:
            
        print('iteration {}'.format(step))    
        print('   Validation Acc: {:.5f}'.format(Acc))

iteration 1000
   Validation Acc: 0.87381
iteration 2000
   Validation Acc: 0.90952
iteration 3000
   Validation Acc: 0.92381
iteration 4000
   Validation Acc: 0.94524
iteration 5000
   Validation Acc: 0.95952
iteration 6000
   Validation Acc: 0.95952
iteration 7000
   Validation Acc: 0.96190
iteration 8000
   Validation Acc: 0.95714
iteration 9000
   Validation Acc: 0.96190
iteration 10000
   Validation Acc: 0.97381


In [98]:
saver = tf.train.Saver()
saver.save(sess, '/content/drive/My Drive/online_shoppers_intention.csv/MLmodels/ANN_model_tf1')

'/content/drive/My Drive/online_shoppers_intention.csv/MLmodels/ANN_model_tf1'

In [99]:
tf.reset_default_graph()

In [100]:
X = tf.placeholder(tf.float32, [None, Training_Fold1.shape[1]])
Y = tf.placeholder(tf.float32, [None, 2])

# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers

# Input Layer
W1 = tf.Variable(tf.random_normal([Training_Fold1.shape[1], noOfNeuron]))
b1 = tf.Variable(tf.random_normal([noOfNeuron]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

# Hidden Layer 1
W2 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b2 = tf.Variable(tf.random_normal([noOfNeuron]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

# Hidden Layer 2
W3 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b3 = tf.Variable(tf.random_normal([noOfNeuron]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

# Output Layer
W4 = tf.Variable(tf.random_normal([noOfNeuron, 2]))
b4 = tf.Variable(tf.random_normal([2]))
hypothesis = tf.matmul(L3, W4) + b4


# define cost/loss &optimizer
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [101]:
sess  = tf.Session()
LoadedModel = tf.train.Saver()
LoadedModel.restore(sess, '/content/drive/My Drive/online_shoppers_intention.csv/MLmodels/ANN_model_tf1')

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/online_shoppers_intention.csv/MLmodels/ANN_model_tf1


In [102]:
for i in range(Fold):
    
    s1= 'Data = Training_Fold%d'%(i+1)
    s2= 'Data_Val = Validation_Fold%d'%(i+1)
    exec(s1)
    exec(s2)

    s3 = 'Validation_Acc_Fold%d = sess.run(accuracy, feed_dict={X : Data_Val, Y: Label_Val})'%(i+1)
    exec(s3)

Accuracy_sum = 0

for j in range(Fold):
    
    s3 = 'Accuracy_sum += Validation_Acc_Fold%d'%(j+1)
    exec(s3)

print('[Performance of ANN(tf1) model] \n')
print('Fold 1: {:.5f} \nFold 2: {:.5f} \nFold 3: {:.5f} \nFold 4: {:.5f} \nFold 5: {:.5f} \n\n * Average accuracy : {:.2f}%'
      .format(Validation_Acc_Fold1*100, Validation_Acc_Fold2*100, Validation_Acc_Fold3*100, Validation_Acc_Fold4*100, Validation_Acc_Fold5*100, Accuracy_sum/Fold*100))

[Performance of ANN(tf1) model] 

Fold 1: 98.80952 
Fold 2: 95.23810 
Fold 3: 97.61904 
Fold 4: 97.61904 
Fold 5: 97.61904 

 * Average accuracy : 97.38%
